# Лекция 4.1: Аппроксимация данных

## Аппроксимация

### Постановка задачи
Задача аппроксимации формулируется следующим образом: дано множество обучающих данных  
$$
\{(x_i, y_i)\}_{i=1}^{N}, \quad x_i \in \mathbb{R}^d, \; y_i \in \mathbb{R},
$$
требуется найти функцию $ f: \mathbb{R}^d \to \mathbb{R} $, которая удовлетворяет условию:
$$
y_i \approx f(x_i), \quad \forall i = 1, \dots, N.
$$

Обычно задача решается через минимизацию функции потерь, например, суммы квадратов ошибок:
$$
\min_{f \in \mathcal{F}} \; \sum_{i=1}^{N} \left( y_i - f(x_i) \right)^2,
$$
где $\mathcal{F}$ – множество рассматриваемых моделей. Такой подход позволяет не только "подогнать" модель под данные, но и контролировать обобщающую способность модели через понятия смещения (bias) и дисперсии (variance).

#### Классические методы

Например, можно построить интерполяционный полином Лагранжа для набора точек $\{(x_i, y_i)\}_{i=1}^{N}$ определяется как:
$$
L(x) = \sum_{i=0}^{n} y_i \cdot l_i(x)
$$

где базисные полиномы $ l_i(x) $ определяются как:
$$
l_i(x) = \prod_{\substack{0 \le j \le n \\ j \neq i}} \frac{x - x_j}{x_i - x_j}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f(x: float):
    return np.sin(x) * x/5

In [ ]:
def lagrange_interpolation(x_points, 
                           y_points, 
                           x):
    """
    Интерполяция Лагранжа

    Аргументы:
        x_points (list[float]): Список значений x
        y_points (list[float]): Список значений y
        x (float): Значение x для интерполяции

    Возвращает:
        float: Значение интерполяции
    """
    def basis_polynomial(i, x):
        """
        Базисный полином Лагранжа

        Аргументы:
            i (int): Индекс базисного полинома
            x (float): Значение x для интерполяции

        Возвращает:
            float: Значение базисного полинома
        """
        terms = [
            (x - x_points[j]) / (x_points[i] - x_points[j])
            for j in range(len(x_points)) if j != i
        ]
        return np.prod(terms, axis=0)

    return (sum(y_points[i] * basis_polynomial(i, x) 
                for i in range(len(x_points))))

In [ ]:
N = 10
x_points = np.linspace(0, 5, N)
y_points = f(x_points)

x_new = np.linspace(0, 5, 100)
y_lagrange = lagrange_interpolation(x_points, 
                                    y_points, 
                                    x_new)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(x_points, y_points, 'bo', label='Data')
plt.plot(x_new, y_lagrange, 'r-', label='Interpolation')
plt.plot(x_new, f(x_new), 'g--', label='True function')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Lagrange interpolation')
plt.axis('equal')
plt.legend()
plt.show()

### Универсальная аппроксимационная теорема

Универсальная аппроксимационная теорема утверждает следующее:

Пусть $ f: \mathbb{R}^n \to \mathbb{R} $ – непрерывная функция, определённая на компактном подмножестве $\mathbb{R}^n$. Тогда для любого $\epsilon > 0$ существует нейронная сеть с одним скрытым слоем, использующая нелинейную активационную функцию $\sigma$, такая, что:

$$
\left| f(x) - \sum_{i=1}^{N} a_i \sigma(\langle w_i, x \rangle + b_i) \right| < \epsilon
$$

для всех $ x $ из этого компактного подмножества, где:
- $ N $ – количество нейронов в скрытом слое,
- $ a_i, w_i, b_i $ – параметры сети (веса и смещения),
- $\sigma$ – нелинейная активационная функция, например, сигмоидальная функция.

> Она еще встретится далее в курсе

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(1, 16)
        self.linear2 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.tanh(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
N = 10
x_points = np.linspace(0, 5, N)
y_points = f(x_points)

x_train = torch.tensor(x_points, 
                       dtype=torch.float32).view(-1, 1)
y_train = torch.tensor(y_points, 
                       dtype=torch.float32).view(-1, 1)

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epochs = 10000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

In [ ]:
x_test = torch.tensor(x_new, dtype=torch.float32).view(-1, 1)
y_pred_nn = model(x_test).detach().numpy()

mse_lagrange = np.mean((y_lagrange - f(x_new))**2)
mse_nn = np.mean((y_pred_nn.flatten() - f(x_new))**2)

print(f"MSE Lagrange: {mse_lagrange:.4f}")
print(f"MSE NN: {mse_nn:.4f}")

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(x_points, y_points, 'bo', label='Data')
plt.plot(x_test.numpy(), y_pred_nn, 'r-', label='NN')
plt.plot(x_test.numpy(), f(x_test.numpy()), 'g--', label='Original function')
plt.xlabel('x')
plt.ylabel('y')
plt.title('NN interpolation')
plt.axis('equal')
plt.legend()
plt.show()

### Линейные и нелинейные методы аппроксимации

#### Линейные методы
В линейном случае предполагается, что зависимость между $ x $ и $ y $ может быть описана линейным соотношением:
$$
f(x) = \langle \mathbf{w}, x \rangle + b,
$$
где:
- $\mathbf{w} \in \mathbb{R}^d$ – вектор коэффициентов,
- $b \in \mathbb{R}$ – смещение.

Обучение модели (например, с использованием метода наименьших квадратов) сводится к решению задачи:
$$
\min_{\mathbf{w}, \, b} \; \sum_{i=1}^{N} \left( y_i - (\langle \mathbf{w}, x_i \rangle + b) \right)^2.
$$
Преимущества линейных моделей:
- Простота и хорошая интерпретируемость;
- Низкая вычислительная сложность.

Ограничение заключается в том, что они не способны адекватно аппроксимировать сложные нелинейные зависимости.

#### Нелинейные методы
Чтобы учитывать нелинейности, можно использовать преобразование исходных признаков. Одним из подходов является введение нового отображения $\phi: \mathbb{R}^d \to \mathbb{R}^{d'}$, приводящего к модели:
$$
f(x) = \langle \mathbf{w}, \phi(x) \rangle + b.
$$

Примером является полиномиальная регрессия, где функция $\phi(x)$ включает полиномы входных признаков:
$$
\phi(x) = [1, x, x^2, \dots, x^p],
$$
и модель становится:
$$
f(x) = w_0 + w_1 x + w_2 x^2 + \dots + w_p x^p.
$$
Преимущества нелинейных методов:
- Гибкость в аппроксимации сложных зависимостей;
- Возможность выбора степени нелинейности через параметр $p$.

Недостатки:
- Рост числа параметров может привести к переобучению;
- Снижение интерпретируемости модели.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
class SingleNeuron(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        self.activation = activation

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

In [ ]:
x_points = np.linspace(-10, 10, 100, dtype=np.float32).reshape(-1, 1)

activations = {
    'ReLU': torch.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh
}

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))

for ax, (name, activation) in zip(axes, activations.items()):
    model = SingleNeuron(activation)
    with torch.no_grad():
        y_pred = model(torch.tensor(x_points)).numpy()
    
    ax.plot(x_points, y_pred, label=f'Activation: {name}')
    ax.set_title(f'Activation: {name}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
model = SingleNeuron(torch.relu)

for name, param in model.named_parameters():
    print(f"{name}, {param.data}")

Проверим влияние функций активации на полноценной нейронной сети.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, activation, num_neurons):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(1, num_neurons)
        self.activation = activation
        self.linear2 = nn.Linear(num_neurons, 1)

    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
def f(x):
    return np.sin(x) * x / 5

In [ ]:
x_points = np.linspace(0, 5, 100, dtype=np.float32)
y_points = f(x_points)

x_train = torch.tensor(x_points, dtype=torch.float32).view(-1, 1)
y_train = torch.tensor(y_points, dtype=torch.float32).view(-1, 1)

In [ ]:
def train_and_plot(activation, num_neurons, ax, title):
    model = SimpleNN(activation, num_neurons)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    epochs = 5000
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    y_pred = model(x_train).detach().numpy()

    ax.plot(x_points, y_points, 'g--', label='Оригинальная функция')
    ax.plot(x_points, y_pred, 'r-', label='Аппроксимация')
    ax.set_title(title)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()

In [ ]:
activations = {
    'ReLU': torch.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh
}

num_neurons_list = [1, 5, 10, 50]

In [ ]:
fig, axes = plt.subplots(nrows=len(num_neurons_list), 
                         ncols=3, figsize=(15, 12))

for col, (name, activation) in enumerate(activations.items()):
    for row, num_neurons in enumerate(num_neurons_list):
        train_and_plot(activation, num_neurons, axes[row, col], 
                       f'{name} с {num_neurons} neurons')

plt.tight_layout()
plt.show()